In [1]:
pip install pypandoc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 38 kB/s 
     |████████████████████████████████| 199 kB 54.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=0d2f74934b4330bb5b1bd94a38236f7b445108b393101156616dec8527c0f600
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [3]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark from http://www.apache.org/dist/spark/ 
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net

In [4]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-01-02 06:53:59--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2023-01-02 06:53:59 (10.4 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [6]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-reviews_us_Grocery").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [7]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url= "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz" 
spark.sparkContext.addFile(url)
grocery_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Grocery_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
grocery_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...|2015-08-31 00:00:00|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...| 

In [8]:
# Get the number of rows in the DataFrame.
grocery_data_df.count()

2402458

# Transform the Data

## Create the "review_id_table".

In [9]:
from pyspark.sql.functions import to_date
grocery_review_id_df = grocery_data_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
grocery_review_id_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R26MV8D0KG6QI6|   42521656|B000SAQCWC|     159713740|2015-08-31 00:00:00|
|R1OF8GP57AQ1A0|   12049833|B00509LVIQ|     138680402|2015-08-31 00:00:00|
|R3VDC1QB6MC4ZZ|     107642|B00KHXESLC|     252021703|2015-08-31 00:00:00|
|R12FA3DCF8F9ER|    6042304|B000F8JIIC|     752728342|2015-08-31 00:00:00|
| RTWHVNV6X4CNJ|   18123821|B004ZWR9RQ|     552138758|2015-08-31 00:00:00|
| RIG9AWFOGRDVO|   23649464|B00AL6QBZ6|     681475449|2015-08-31 00:00:00|
|R1S1XSG4ZCHDGS|   32778285|B00BCNSTRA|     578681693|2015-08-31 00:00:00|
| RB15NBVY5ELVW|   46612941|B0089MM2BG|     350968436|2015-08-31 00:00:00|
| R56358YM1ZJ7I|   31525399|B00Y1C9770|     729982780|2015-08-31 00:00:00|
|R1ODXB3C9UP3NL|   19624355|B00J074W94|       2499702|2015-08-31 00:00:00|
|R155ZMVLD5C9BP|   319103

## Create the "products" Table

In [10]:
grocery_products_df = grocery_data_df.dropDuplicates(subset= ["product_id","product_title"] )

In [11]:
grocery_products_df.count()

305538

In [12]:
grocery_products_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   11425554|R263F2Q3CR0JJG|0805470867|     518591127|Communion Bread P...|         Grocery|          5|            0|          2|   N|                Y|             WORSHIP|The communion bre...|2013-05-18 00:00:00|
|         US|   25785649| R17QYNM2M4AES|0950000019|     896968165|     Royal Manna Mix| 

## Create the "customers" Table

In [13]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
grocery_customer_df = grocery_data_df.groupBy('customer_id').agg({"customer_id": "count"})
grocery_customer_df = grocery_customer_df.withColumnRenamed("count(customer_id)", "customer_review_count")
grocery_customer_df.show()

+-----------+---------------------+
|customer_id|customer_review_count|
+-----------+---------------------+
|   21294263|                    1|
|    8619125|                    1|
|   43626894|                    1|
|   38209321|                    4|
|   40568205|                    1|
|   13480593|                    1|
|   19505799|                    1|
|   43150161|                    2|
|   28802315|                    2|
|   35329257|                    1|
|   49101505|                    1|
|    1117644|                    1|
|   28377689|                    1|
|   23493243|                    1|
|   48670265|                    1|
|   49041516|                    3|
|   37499049|                    2|
|   10920752|                    1|
|    5627998|                    1|
|     135423|                    1|
+-----------+---------------------+
only showing top 20 rows



In [14]:
grocery_customer_df.count()

1363986

## Create the "vine_table".

In [15]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
from pyspark.sql.functions import to_date
grocery_vine_df = grocery_data_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
grocery_vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R26MV8D0KG6QI6|          5|            0|          0|   N|
|R1OF8GP57AQ1A0|          5|            0|          0|   N|
|R3VDC1QB6MC4ZZ|          5|            0|          0|   N|
|R12FA3DCF8F9ER|          5|            0|          0|   N|
| RTWHVNV6X4CNJ|          5|            0|          0|   N|
| RIG9AWFOGRDVO|          2|            1|          1|   N|
|R1S1XSG4ZCHDGS|          5|            1|          1|   N|
| RB15NBVY5ELVW|          5|            2|          2|   N|
| R56358YM1ZJ7I|          5|            0|          0|   N|
|R1ODXB3C9UP3NL|          1|            1|          3|   N|
|R155ZMVLD5C9BP|          4|            3|          4|   N|
|R2IXW43PUNYM0G|          4|            1|          1|   N|
|R3LYEG1QCK2BG0|          5|            0|          0|   N|
|R20LREICPM3YH0|          5|            

# Load

In [16]:
mode = "append"
jdbc_url="jdbc:postgresql://database-3.c2psmop0qavn.ap-southeast-2.rds.amazonaws.com:5432/groceries_db" 
config = {"user":"", "password": "", "driver":"org.postgresql.Driver"}

In [17]:
# Write review_id_df to table in RDS
grocery_review_id_df.write.jdbc(url=jdbc_url, table='grocery_review_id', mode=mode, properties=config)

In [18]:
# Write products_df to table in RDS
grocery_products_df.write.jdbc(url=jdbc_url, table='grocery_product', mode=mode, properties=config)

In [19]:
# Write customers_df to table in RDS
grocery_customer_df.write.jdbc(url=jdbc_url, table='grocery_customers', mode=mode, properties=config)

In [20]:
# Write vine_df to table in RDS
grocery_vine_df.write.jdbc(url=jdbc_url, table='grocery_vine', mode=mode, properties=config)